<hr style="margin-bottom: 50px;">
<center>
    <h1 style="margin-top: 0; margin-bottom: 0;">
        <b><u>Gradient Boosting</u></b>
    </h1>
</center>
<hr style="margin-top: 50px;">


In [32]:
import config, src
import numpy as np
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
data = src.load_data(config.PROCESSED_DATA_COMBINED_RAP_HIPHOP_FILE)

X_train, X_test, y_train, y_test = src.split_data(data, target_column='music_genre')

class_weight = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weight = dict(zip(np.unique(y_train), class_weight))

Dans cette partie on se propose d'appliquer les méthodes de classifications de types séquentielles du Boosting. Les méthodes de Boosting remontent á l'article de Friedman sur l'approximation de fonctions "GREEDY" par une optimisation numérique de fonctions. Divers algorithmes tels que AdaBOOST ,XGBOOST ont été developpés ces derniéres années et se sont révéles particulièrement efficaces. On utilise l' estimateur  HistGradientBoostingClassifier car il a l'avantage d'être plus rapide que GradientBoostingClassifier\(\).  



In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score,make_scorer

Gbd=HistGradientBoostingClassifier(learning_rate=0.1,max_iter=100)

Gbd.fit(X_train,y_train)

src.evaluate_model(Gbd, X_test, y_test, classes=config.CLASSES_COMBINED_RAP_HIPHOP)

In [ ]:
from  xgboost import XGBClassifier

Xgb=XGBClassifier(n_estimators=150,learning_rate=0.1)

Xgb.fit(X_train,y_train)

src.evaluate_model(Xgb, X_test, y_test, classes=config.CLASSES_COMBINED_RAP_HIPHOP)

On réalise une GridSearch pour améliorer nos résultats.


from sklearn.metrics import make_scorer, accuracy_score
from sklearn.experimental import enable_hist_gradient_boosting  
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np

param_grid = {
    'learning_rate': np.arange(0.05, 1.5, 0.25),
    'max_iter': np.arange(50,200,10)
}


Gbd = HistGradientBoostingClassifier()

scoring = make_scorer(accuracy_score, greater_is_better=True)


grid_search = GridSearchCV(Gbd, param_grid, cv=5, scoring=scoring)
grid_search.fit(X_train, y_train)


scores = -grid_search.cv_results_['mean_test_score']
scores = scores.reshape(len(param_grid['max_iter']), len(param_grid['learning_rate']))

print("Best Hyperparameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_
src.evaluate_model(best_model, X_test, y_test, classes=config.CLASSES_COMBINED_RAP_HIPHOP)

---